In [1]:
## IMPORT MODULES
import pandas as pd
import numpy as np
import os,time,fnmatch

In [2]:
## CREATE LINK TO DATAPATH MODULE
import sys
sys.path.insert(0,'/home/bhabas/catkin_ws/src/crazyflie_simulation/src/crazyflie_gazebo_sim/src/utility')
from data_analysis import DataFile
dataPath = '/home/bhabas/catkin_ws/src/crazyflie_simulation/local_files/data/'

In [3]:
## CREATE AN ARRAY FOR ALL TEST COMBINATIONS
temp_list = []
for vz_d in np.arange(1.5,4.0,0.5): # [1.5,3.5]
    for vx_d in np.arange(0,3.5,0.5):   # [0,3.0]
        # print(f"{vz_d}|{vx_d}")
        temp_list.append([vz_d,vx_d])
test_arr = np.asarray(temp_list)
test_arr

array([[1.5, 0. ],
       [1.5, 0.5],
       [1.5, 1. ],
       [1.5, 1.5],
       [1.5, 2. ],
       [1.5, 2.5],
       [1.5, 3. ],
       [2. , 0. ],
       [2. , 0.5],
       [2. , 1. ],
       [2. , 1.5],
       [2. , 2. ],
       [2. , 2.5],
       [2. , 3. ],
       [2.5, 0. ],
       [2.5, 0.5],
       [2.5, 1. ],
       [2.5, 1.5],
       [2.5, 2. ],
       [2.5, 2.5],
       [2.5, 3. ],
       [3. , 0. ],
       [3. , 0.5],
       [3. , 1. ],
       [3. , 1.5],
       [3. , 2. ],
       [3. , 2.5],
       [3. , 3. ],
       [3.5, 0. ],
       [3.5, 0.5],
       [3.5, 1. ],
       [3.5, 1.5],
       [3.5, 2. ],
       [3.5, 2.5],
       [3.5, 3. ]])

In [11]:
## CREATE DATAFRAME FROM ALL TESTS

d = []                                                  # Create empty list
for vz_d,vx_d in test_arr:                              # Iter over all combinations in test_arr
    for file in os.listdir(dataPath):                   # Iter over all files in dir 
        csv = os.path.join(dataPath,file)               #   and join name w/ base path

        if fnmatch.fnmatch(file,f"Vz_{vz_d}--Vx_{vx_d}--trial_*.csv"): # If filename matched selected IC 

            trial = DataFile(csv)                       # Create object called trial

            trial_num = file[-5]                        # Retrieve trial num
            landing_rate = trial.landing_rate()         # Retrieve landing success rate
            policy,sigma = trial.grab_finalPolicy()     # Retrieve final policy and sigma

            RREV_trigger,G1,G2 = policy                 # Break up policy
            RREV_sig,G1_sig,G2_sig = sigma              # Break up sigma

            d.append((vz_d,vx_d,trial_num,landing_rate, # Append values to list
                RREV_trigger,G1,G2,
                RREV_sig,G1_sig,G2_sig))

policy_df = pd.DataFrame(d, columns=( # Convert list to pd.dataframe
    'vz_d','vx_d','trial_num','landing_rate',
    'RREV_trigger','G1','G2',
    'RREV_sig','G1_sig','G2_sig'))
policy_df

,vz_d,vx_d,trial_num,landing_rate,RREV_trigger,G1,G2,RREV_sig,G1_sig,G2_sig
0,1.5,0.0,1,0.00,4.11,6.18,3.78,0.07,0.38,0.28
1,1.5,0.0,0,0.00,5.00,7.59,3.88,0.83,0.27,0.11
2,1.5,0.0,2,0.00,4.82,5.82,3.44,0.08,0.15,0.28
3,1.5,0.5,2,0.00,4.33,7.39,3.25,0.10,0.19,0.07
4,1.5,0.5,1,0.00,4.55,6.98,3.86,0.13,0.10,0.14
...,...,...,...,...,...,...,...,...,...,...
89,3.5,2.5,1,0.75,5.16,7.81,1.81,0.16,0.05,0.52
90,3.5,2.5,2,0.00,5.16,12.89,2.43,0.03,1.74,0.51
91,3.5,3.0,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00
92,3.5,3.0,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [12]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
%matplotlib widget


In [17]:
## POLICY AVERAGED DATAFRAME

# Group dataframe values by IC and find avg/std of their values 
policy_avg_df = policy_df.groupby(['vz_d','vx_d'],as_index=False).agg([np.mean,np.std])

In [24]:
## LANDING RATE DATA
LR_df = policy_avg_df.landing_rate.reset_index()
# landing_rate_df

In [33]:
## LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.plot_trisurf(LR_df['vz_d'],LR_df['vx_d'],LR_df['mean'],cmap=cm.jet)
ax.set_xlabel('vz_d')
ax.set_ylabel('vx_d')
ax.set_zlabel('Avg LR %')
ax.set_title('Average Landing Rate for Final Two Episodes')



ax.xaxis.set_ticks(np.arange(1.5,4.0,0.5))
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
# policy_df.groupby(['vz_d','vx_d'])['RREV_trigger'].mean().reset_index()
grouped = policy_df.groupby(['vz_d','vx_d'],as_index=False).agg([np.mean,np.std])
grouped['landing_rate','mean'].reset_index()

,vz_d,vx_d,landing_rate
,,,mean
0,1.5,0.0,0.000000
1,1.5,0.5,0.000000
2,1.5,1.0,0.016667
3,1.5,1.5,0.000000
4,1.5,2.0,0.000000
5,1.5,2.5,0.000000
6,1.5,3.0,0.000000
7,2.0,0.0,0.016667
8,2.0,0.5,0.000000


In [26]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vz_d'],policy_df['vx_d'],policy_df['RREV_trigger'])

ax.set_xlabel('vz_d')
ax.set_ylabel('vx_d')
ax.set_zlabel('RREV')

ax.xaxis.set_ticks(np.arange(1.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vz_d'],policy_df['vx_d'],policy_df['G2'])

ax.set_xlabel('vz_d')
ax.set_ylabel('vx_d')
ax.set_zlabel('G2')

ax.xaxis.set_ticks(np.arange(1.5,4.0,0.5))
plt.show()